In [196]:
import pandas as pd
import numpy as np
import wbgapi as wb
from pandas_profiling import ProfileReport

In [197]:
#indice de capital humain
df_hai = pd.read_excel('../data-series-retrospectives-de-l-indice-hai.xlsx').drop('iso_3', axis = 1)

df_hai = pd.wide_to_long(df_hai, ['HAI'], i= 'country', j = 'year').reset_index()# de 1990 à 2014

# On harmonise le nom de certains pays
cntry_uni = [('Iran (Islamic Republic of)','Iran'), ("Bolivia (Plurinational State of)",'Bolivia'), ("South \u200eSudan",'South Sudan'), ( "Venezuela (Bolivarian Republic of)",'Venezuela'),("Côte D'Ivoire","Cote D'Ivoire"),("Syrian Arab Republic","Syria")]

for cntry1,cntry2 in cntry_uni: 
    df_hai[df_hai.country == cntry1] = df_hai[df_hai.country == cntry1].assign(country=cntry2)

In [198]:
# Données sur l'instabilité politique

#df_insta_pol = pd.read_excel(r'C:\Users\matth\OneDrive - GENES\ENSAE 2A\Python pour le data scientist/p5v2018.xls')
df_insta_pol = pd.read_excel('../p5v2018.xls')


df_insta_pol = df_insta_pol[df_insta_pol['year']>= 1990][['country','year','polity2']] #de 1990 à 2018

# On harmonise le nom de certains pays
cntry_uni = [('Myanmar (Burma)','Myanmar')]

for cntry1,cntry2 in cntry_uni: 
    df_insta_pol[df_insta_pol.country == cntry1] = df_insta_pol[df_insta_pol.country == cntry1].assign(country=cntry2)

In [199]:
#vulnérabilité économique structurelle
#df1 = pd.read_excel(r'C:\Users\matth\OneDrive - GENES\ENSAE 2A\Python pour le data scientist/indicateur-de-vulnerabilite.xlsx')
df_eco = pd.read_excel('../indicateur-de-vulnerabilite.xlsx')[['country','year','EVI']]


df_eco = df_eco[df_eco['year']>=1990]

# On harmonise le nom de certains pays
cntry_uni = [('Syrian Arab Republic','Syria'),('Yemen, Rep.','Yemen'),('Venezuela, RB','Venezuela'),('Iran, Islamic Rep.','Iran'),('Egypt, Arab Rep.','Egypt')]

for cntry1,cntry2 in cntry_uni: 
    df_eco[df_eco.country == cntry1] = df_eco[df_eco.country == cntry1].assign(country=cntry2)
    

In [200]:
# Création d'une deuxième base comportant des variables par pays et année que l'on utilisera dans la partie modélisation

#Population totale, PIB par habitant (en $), Indice GINI, Croissance annuelle du PIB, taux de chomage, mortalité infantile, indice de la valeur des exportations, IDE
df_wb= wb.data.DataFrame(['SP.POP.TOTL','NY.GDP.PCAP.KD','NY.GDP.MKTP.KD.ZG','SH.DTH.IMRT','SI.POV.GINI', 'SL.UEM.TOTL.NE.ZS','TX.VAL.MRCH.XD.WD', 'BM.KLT.DINV.CD.WD'], labels =True, time = range(1990, 2018), columns = 'series').reset_index().rename(columns = {'Time': 'year', 'SP.POP.TOTL' : 'population', 'NY.GDP.PCAP.KD' : 'PIB/hab', 'NY.GDP.MKTP.KD.ZG' : 'croissancePIB', 'SH.DTH.IMRT' : 'mortalite-enf', 'SI.POV.GINI' : 'gini', 'SL.UEM.TOTL.NE.ZS' : 'chomage', 'TX.VAL.MRCH.XD.WD' : 'export', 'BM.KLT.DINV.CD.WD' : 'IDE'}).drop(columns = ['time','economy'])

In [202]:
df_wb.rename(columns = {'Country':'country'}, inplace = True)
df_wb.year = df_wb.year.astype(int)

# On harmonise le nom de certains pays
cntry_uni = [('Syrian Arab Republic','Syria'),('Yemen, Rep.','Yemen'),('Iran, Islamic Rep.','Iran'),('Egypt, Arab Rep.','Egypt')]

for cntry1,cntry2 in cntry_uni: 
    df_wb[df_wb.country == cntry1] = df_wb[df_wb.country == cntry1].assign(country=cntry2)

In [206]:
# On s'assure que les pays sur lesquelles on va faire la jointure soient bien les mêmes dans les df
cntry_list = df_hai.country.unique()

data_wb = df_wb[df_wb.country.isin(cntry_list)]
data_eco = df_eco[df_eco.country.isin(cntry_list)]
data_pol = df_insta_pol[df_insta_pol.country.isin(cntry_list)]

cntry_list2 = data_pol.country.unique()

data_wb = data_wb[data_wb.country.isin(cntry_list2)]
data_eco = data_eco[data_eco.country.isin(cntry_list2)]
data_hai = df_hai[df_hai.country.isin(cntry_list2)]

cntry_list3 = data_wb.country.unique()

data_eco = data_eco[data_eco.country.isin(cntry_list3)][data_eco.year <= 2014]
data_pol = data_pol[data_pol.country.isin(cntry_list3)][data_pol.year <= 2014]
data_hai = df_hai[df_hai.country.isin(cntry_list3)]

<ipython-input-206-ba46e5934d2a>:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_eco = data_eco[data_eco.country.isin(cntry_list3)][data_eco.year <= 2014]
<ipython-input-206-ba46e5934d2a>:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_pol = data_pol[data_pol.country.isin(cntry_list3)][data_pol.year <= 2014]


In [215]:
df_merge1 = pd.merge(data_wb, data_eco, on=['country', 'year'])
df_merge2 = pd.merge(df_merge1, data_pol, on=['country', 'year'])
df_merge3 = pd.merge(df_merge2, data_hai, on=['country', 'year'])

In [217]:
# L'indicateur utilisé est la proportion qu'occupe un pays donné dans le nombre total de mort au cours d'une année donnée

df_ucdp = pd.read_csv('../UCDP.csv')

def ratio_best(country,y):
    # Nombre d'années que le conflit est en activité durant jusqu'à l'année 
        
    return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()

df_indicator = pd.DataFrame({'country':[],
        'year': [],
        'ratio':[]})

for y in df_ucdp['year'].unique():
    for cntry in df_ucdp[df_ucdp['year'] == y]['country'].unique():

        df_indicator = df_indicator.append({'country':cntry,'year':y,'ratio':ratio_best(cntry,y)},ignore_index=True)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum

<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: User

<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: User

<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: User

<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: User

...

<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: User

<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: User

<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: User

<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: User

<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()
<ipython-input-217-1f7da4dd115b>:8: User

In [221]:
# On s'assure que les pays sur lesquelles on va faire la jointure soient bien les mêmes dans les df
 
cntry_list = df_merge3.country.unique()

data_indicator = df_indicator[df_indicator.country.isin(cntry_list)][df_indicator.year >= 1990][df_indicator.year <= 2014]
df_merge3 = df_merge3[df_merge3.country.isin(data_indicator.country.unique())]

<ipython-input-221-06e7a1ccd090>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_indicator = df_indicator[df_indicator.country.isin(cntry_list)][df_indicator.year >= 1990][df_indicator.year <= 2014]
<ipython-input-221-06e7a1ccd090>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_indicator = df_indicator[df_indicator.country.isin(cntry_list)][df_indicator.year >= 1990][df_indicator.year <= 2014]


In [228]:
df_merge4 = pd.merge(data_indicator, df_merge3, on=['country', 'year'])

In [254]:
df_border = pd.read_csv('../border.csv')
df_border.country_name.unique()

# On harmonise le nom de certains pays
cntry_uni = [('Bolivia (Plurinational State Of)','Bolivia'), ("Iran (Islamic Republic of)",'Iran'), ("Venezuela (Bolivarian Republic of)",'Venezuela')]

for cntry1,cntry2 in cntry_uni: 
    df_border[df_border.country_name == cntry1] = df_border[df_border.country_name == cntry1].assign(country_name=cntry2)
    df_border[df_border.country_border_name == cntry1] = df_border[df_border.country_border_name == cntry1].assign(country_border_name=cntry2)

In [270]:
def getregion_ratio(country,year):
    neighbors = df_border[df_border.country_name == country].country_border_name.values
    return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()

In [271]:
df_merge4['ratio_zone'] = df_merge4.apply(lambda x: getregion_ratio(x['country'], x['year']), axis=1)

<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indica

<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indica

<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indica

<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indica

<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indica

...

<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indica

<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indica

<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indica

<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indica

<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indicator[data_indicator.year == year][data_indicator.country.isin(neighbors)].ratio.sum()
<ipython-input-270-8fdf2fb7f331>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_indica

In [272]:
df_merge4

,country,year,ratio,IDE,croissancePIB,PIB/hab,mortalite-enf,gini,chomage,population,export,EVI,polity2,HAI,ratio_zone
0,Afghanistan,1990.0,0.015885,NaN,NaN,NaN,75591.0,NaN,NaN,12412311.0,127.081081,31.802521,-8.0,15.131342,0.004606
1,Algeria,1990.0,0.000010,4.688804e+06,0.800001,3123.995683,34550.0,NaN,21.26,25758872.0,67.930716,18.645416,-2.0,66.503784,0.008404
2,Angola,1990.0,0.027374,9.000000e+05,-3.450099,2998.780751,75621.0,NaN,NaN,11848385.0,49.034746,32.735638,-7.0,13.832288,0.000336
3,Bangladesh,1990.0,0.000986,5.000000e+05,5.622258,512.096589,354809.0,NaN,NaN,103171957.0,26.118075,35.014923,-5.0,24.284822,0.049669
4,Botswana,1990.0,0.000010,7.363747e+06,6.772822,3691.632890,1604.0,NaN,NaN,1286756.0,65.782529,NaN,7.0,57.373341,0.015644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,Sudan,2001.0,0.099949,NaN,6.500360,1053.684692,69829.0,NaN,NaN,27971077.0,94.038855,46.889923,-6.0,44.879780,0.022414
1074,Thailand,2001.0,0.000672,4.277032e+08,3.444249,3605.368235,16054.0,NaN,2.60,63539190.0,94.207722,25.436676,9.0,80.320534,0.000000
1075,Turkey,2001.0,0.002876,4.970000e+08,-5.750007,6075.995627,39976.0,NaN,8.38,64192243.0,112.814087,13.717976,7.0,86.371521,0.004676
1076,Uganda,2001.0,0.005698,NaN,5.183661,524.790767,96369.0,NaN,NaN,24388974.0,111.842145,35.405231,-4.0,36.782001,0.060093


In [273]:
# On exporte les données rassemblées 

df_merge4.to_csv('../df_regression.csv', index=False)
#df_merge3.to_csv(r'C:\Users\matth\OneDrive - GENES\ENSAE 2A\Python pour le data scientist/df_regression.csv', index=False)